In [ ]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

import plotly.io as pio
import os
import optuna

from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend

In [ ]:
# ----------------------------------#
#    View best values in journal    #
# ----------------------------------#

study_path = os.path.join("study/cifar10-baseline-2-c3b5ae7b140f363a", "journal.log")
storage = JournalStorage(JournalFileBackend(study_path))

# Show all summaries in this journal
study_summaries = optuna.get_all_study_summaries(storage=storage)
for summary in study_summaries:
    print(summary.study_name, summary.n_trials, summary.best_trial.value)

# Load the study
study_name = "cifar10-baseline-2"
study = optuna.load_study(storage=storage, study_name=study_name)
study.best_trial.value

print("Best trial:")
best_trial = study.best_trial
print(f"Value: {best_trial.value}")
print("Params:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

In [ ]:
# -----------------------------------#
#    Visualize Journal With Plots    #
# -----------------------------------#
pio.renderers.default = 'iframe_connected'

In [ ]:
plot_optimization_history(study).show()

In [ ]:
plot_intermediate_values(study).show()

In [ ]:
plot_parallel_coordinate(study).show()

In [ ]:
plot_contour(study).show()

In [ ]:
plot_slice(study).show()

In [ ]:
plot_param_importances(study).show()

In [ ]:
plot_edf(study).show()

In [ ]:
plot_rank(study).show()

In [ ]:
plot_timeline(study).show()

In [ ]:
# ----------------------------------------#
#    Visualize Journal With Dataframes    #
# ----------------------------------------#
# ----------------------------
# Extract completed trials only
# ----------------------------
df = study.trials_dataframe(attrs=("params", "value", "state"))
df = df[df["state"] == "COMPLETE"].dropna(subset=["value"])

X_raw = df.filter(like="params_")
y = df["value"].values

if X_raw.empty:
    raise ValueError("No completed trials found for GP fitting.")

# ----------------------------
# Apply log10 scaling to selected parameters
# ----------------------------
X_log_scaled = X_raw.copy()
for col in X_log_scaled.columns:
    if col.endswith("lr") or col.endswith("weight_decay"):
        X_log_scaled[col] = np.log10(X_log_scaled[col])

# ----------------------------
# Normalize hyperparameters
# ----------------------------
scaler = MinMaxScaler()
X = scaler.fit_transform(X_log_scaled)

# ----------------------------
# Fit Gaussian Process
# ----------------------------
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y)

# ----------------------------
# Predict mean and std for each completed trial
# ----------------------------
y_mean, y_std = gp.predict(X, return_std=True)

# ----------------------------
# Create output DataFrame
# ----------------------------
output_df = X_raw.copy()
output_df["gp_mean"] = y_mean
output_df["gp_std"] = y_std
output_df["observed_accuracy"] = y

# Sort by predicted mean accuracy
output_df_sorted = output_df.sort_values(by="gp_mean", ascending=False)

# Show top 5
output_df_sorted.head(5)